In [10]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Date, Numeric, DateTime
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy.ext.automap import automap_base

In [11]:
# The code was removed by Watson Studio for sharing.

In [12]:
#Connect to DB2
connection_string = "db2+ibm_db://" + user + ":" + pwd + "@dashdb-txn-flex-yp-dal13-43.services.dal.bluemix.net:50000/BLUDB"


In [13]:

engine = create_engine(connection_string, echo=False)
meta = MetaData()

nome_tabela = 'COVID'

covid = Table('covid', meta, 
    Column("DATE", Date),
    Column("TYPE", String(30)),
    Column("CASES", Integer),
    Column("DIFFERENCE", Integer),
    Column("COUNTRY", String(50)),
    Column("PROVINCE_STATE", String(50)), 
    Column("LAT", Numeric(18,8)),
    Column("LONG", Numeric(18,8)),
    Column("LATEST_DATE", DateTime))

if not engine.dialect.has_table(engine, nome_tabela, schema = 'BLUADMIN'):
    meta.create_all(engine)
else:
    conn = engine.connect()
    #stmt = covid.delete().where(students.c.lastname == 'Khanna')
    stmt = covid.delete()
    conn.execute(stmt)
    conn.close()

In [14]:
Base = automap_base()

#engine = create_engine(connection_string, echo=True)
session = scoped_session(sessionmaker(autocommit=True,
                                      autoflush=False,
                                      bind=engine))

Base.prepare(engine, reflect=True)
base_sa = Base.classes

In [15]:
df = pd.read_csv('https://download.data.world/s/3xpcwdh7es3o6uqyztkkhdnh4ht7i7')
df.head()

,DATE,TYPE,CASES,DIFFERENCE,COUNTRY,PROVINCE_STATE,LAT,LONG,LATEST_DATE
0,2020-06-04,Confirmed,3262,152,Cote d'Ivoire,NaN,7.540000,-5.547100,2020-06-04T23:15:39
1,2020-06-04,Deaths,0,0,Eritrea,NaN,15.179400,39.782300,2020-06-04T23:15:39
2,2020-06-04,Deaths,1,0,France,French Guiana,4.000000,-53.000000,2020-06-04T23:15:39
3,2020-06-04,Confirmed,298,0,Canada,Manitoba,53.760900,-98.813900,2020-06-04T23:15:39
4,2020-06-04,Confirmed,20,0,France,New Caledonia,-20.904305,165.618042,2020-06-04T23:15:39


In [16]:
nome_tabela = 'covid'

In [17]:
df.to_sql(name=nome_tabela, con=engine, if_exists='append', index=False)
#df.to_sql(name=nome_tabela, con=engine, if_exists='append', index=False, chunksize=1000)   # With number of records to be processed each time

In [18]:
df2 = pd.read_sql_table(nome_tabela, engine)
df2.head()

,DATE,TYPE,cases,difference,country,province_state,lat,LONG,latest_date
0,2020-06-04,Deaths,22,0,New Zealand,None,-40.900600,174.886000,2020-06-04 23:15:39
1,2020-06-04,Confirmed,82,8,Suriname,None,3.919300,-56.027800,2020-06-04 23:15:39
2,2020-06-04,Confirmed,1288,115,Central African Republic,None,6.611100,20.939400,2020-06-04 23:15:39
3,2020-06-04,Deaths,291,0,Israel,None,31.046051,34.851612,2020-06-04 23:15:39
4,2020-06-04,Deaths,1770,82,Pakistan,None,30.375300,69.345100,2020-06-04 23:15:39


In [19]:
session.close()